In [ ]:
import os
import cv2
import pathlib
from tqdm import tqdm

def crop_sides_folder(input_dir, output_dir, crop_ratios, exts=(".png", ".jpg", ".jpeg", ".tif", ".tiff")):
    input_dir = pathlib.Path(input_dir)
    output_dir = pathlib.Path(output_dir)
    files = [p for p in input_dir.rglob("*") if p.suffix.lower() in exts]

    for path in tqdm(files, desc="Cropping images"):
        rel = path.relative_to(input_dir)
        out_path = output_dir / rel
        out_path.parent.mkdir(parents=True, exist_ok=True)

        img = cv2.imread(str(path))
        if img is None:
            print(f"[WARN] Skipping unreadable: {path}")
            continue

        h, w = img.shape[:2]
        left = int(w * crop_ratios.get("left", 0))
        right = int(w * crop_ratios.get("right", 0))
        top = int(h * crop_ratios.get("top", 0))
        bottom = int(h * crop_ratios.get("bottom", 0))
        cropped = img[top:h - bottom, left:w - right]
        cv2.imwrite(str(out_path), cropped)

    print(f"\n✅ Done! Cropped {len(files)} images → {output_dir}")

from google.colab import drive
drive.mount('/content/drive')

input_dir = input("Enter input folder path: ").strip()
output_dir = input("Enter output folder path: ").strip()

print("\nEnter cropping percentage for each side (0–100). Example: 5 means 5% crop.")
left = float(input("Crop from LEFT (%): ") or 0) / 100
right = float(input("Crop from RIGHT (%): ") or 0) / 100
top = float(input("Crop from TOP (%): ") or 0) / 100
bottom = float(input("Crop from BOTTOM (%): ") or 0) / 100

crop_ratios = {"left": left, "right": right, "top": top, "bottom": bottom}
crop_sides_folder(input_dir, output_dir, crop_ratios)
